# Tweepy Package Expo Group 44


First, ensure that you have installed Tweepy using the following commmand:

In [ ]:
pip install tweepy

Next, import Tweepy, Pandas, and a few other packages we will need to work with

In [ ]:
import tweepy
import webbrowser
import time 
import pandas as pd

## Twitter API Authentication Credentials

The Twitter API requires that all requests use OAuth to authenticate, so you will need the following 4 credentials to use the API: Consumer Key, Consumer Secret, Acess Token, Access Secret. In order to gain access to these credentials, use these steps:
1. Apply for a Twitter developer account at the folllwing link: https://developer.twitter.com/en . Make sure you state that you are a student using this for educational purposes.
2. Once you get a developer account, you will need to create an app within your developer account. Twitter grants authentication credentials to apps, not users or accounts. An app can be any tool or bot that uses the Twitter API. For our purposes, we are creating an app called "Tweepy Package Expo"
3. Once you create your app, click the "Details" button of the app to find the "Keys and Tokens" tab where you can generate and copy the key, token, and secrets to use in your code. Make sure to save these somewhere on your computer so you don't have to generate new credentials every time. Store your Consumer Key and Consumer Secret as variables to be used in the OAuth handler like as follows:

In [ ]:
consumer_key = "Yz9KyQNKUEU25mZrc7XuvHUnQ"
consumer_secret = "Y2QTFiAyuZNQmjwDZUKCXzZRi1F3RS3PnmI8jfDl7MPBOCNFwA"

callback_uri = 'oob'

In [ ]:
#Authenticate with Twitter
auth = tweepy.OAuthHandler(consumer_key,consumer_secret, callback_uri)

#Authenticates our credentials and grabs redirect url to send user to
redirect_url = auth.get_authorization_url()

In [ ]:
#Opens webbrowser to authenticate from Twitter
webbrowser.open(redirect_url)

The previous command opens a web browser that will ask you to allow Tweepy to gain full access to your Developer Twitter account. Click "Authorize App" and then store the pin that shows next as "User Pin Input"

In [ ]:
user_pin_input = input("What is the pin? ")

In [ ]:
auth.get_access_token(user_pin_input)

Call the API using the following statement and input the auth object that contains your consumer key and consumer secret

In [ ]:
api = tweepy.API(auth)

.me()  is a Tweepy method that is used to get the authenicated user's information

In [ ]:
me = api.me()


# Methods for Tweets

The following statements create a home_timeline object called "timeline" that iterates through every username and tweet of our timeline: 
- update_status() is used to create a new tweet from a Python String
- home_timeline() is used to pull the last 20 entries from your timeline

In [ ]:
timeline = api.home_timeline()

for tweet in timeline:
    print(f"{tweet.user.name} said {tweet.text}")

In [ ]:
api.update_status("Good morning MIST 5730")

# Follow Users using create_friendship() method
Parameters:
- screen_name: Specifies the screen name of the user. Helpful for disambiguating when a valid screen name is also a user ID.
- user_id: Specifies the ID of the user. Helpful for disambiguating when a valid user ID is also a valid screen name.
- follow: Enable notifications for the target user in addition to becoming friends.

Returns User object

In [ ]:
first_follow = api.create_friendship("microsoft")
print(first_follow)

In [ ]:
second_follow = api.create_friendship("elonmusk")
print(second_follow)

## Return top 50 trends based on location with trends_place() method

Parameters:
- id : The Yahoo! Where On Earth ID of the location to return trending information for. Global information is available by using 1 as the WOEID.
- exclude : Setting this equal to hashtags will remove all hashtags from the trends

Returns JSON object

In [ ]:
atlantaID = 2357024

trends = api.trends_place(atlantaID)
print(trends)

In [ ]:
import pandas as pd

In [ ]:
df = pd.json_normalize(trends, record_path = ['trends'])
df

## Follow and Reply to Mentions Bot

Create a "bot" that replies to users who Tweet certain key words and follows them if you are not already.

In [ ]:
import logging
import os

logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)


#Create check mentions function
def check_mentions(api, keywords, since_id):
    logger.info("Retrieving mentions")
    new_since_id = since_id
    for tweet in tweepy.Cursor(api.mentions_timeline,
        since_id=since_id).items():
        new_since_id = max(tweet.id, new_since_id)
        if tweet.in_reply_to_status_id is not None:
            continue
            #if keyword is in tweet, console will print answering to ____ and continue
        if any(keyword in tweet.text.lower() for keyword in keywords):
            logger.info(f"Answering to {tweet.user.name}")
            
            #follows user tweeting at account if not following already
            if not tweet.user.following:
                tweet.user.follow()
            
            #actualy reply to tweet
            api.update_status(
                status="MIST 5730 is a great course you should take with Dr. Safadi at UGA. That's where I was created.",
                in_reply_to_status_id=tweet.id,
                auto_populate_reply_metadata = True
            )
    return new_since_id

def main():
    since_id = 1
    while True:
        #set keywords that we will check our mentions for 
        since_id = check_mentions(api, ["python", "learn"], since_id)
        logger.info("Waiting...")
        time.sleep(15)

if __name__ == "__main__":
    main()